In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Sentence-context embeddings for a list of target words — LAST LAYER ONLY.

For each word:
  1) Build a template sentence: "What is the meaning of the word {word}?"
  2) Run through the model with output_hidden_states=True
  3) Locate token span(s) of the word inside the sentence
  4) Pool subword vectors **from the last hidden layer only**
  5) Save a single vector of shape (hidden_size,) as .npy

Output: OUT_DIR / <model_short> / "sentence_last" / "<word>.npy"
"""

import gc
import json
from pathlib import Path
from typing import List, Tuple

import numpy as np
import pandas as pd
import torch
from transformers import AutoConfig, AutoTokenizer, AutoModel, AutoModelForCausalLM

# ---------------- CONFIG ----------------
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
USE_BF16 = torch.cuda.is_available()

POOL = "mean"  # "mean" | "first" | "last"  (for subword pooling within the span)
SENTENCE_TEMPLATE = "What is the meaning of the word {word}?"

WORDS_CSV = "/Users/tikhomirova/PycharmProjects/Paper ICRL/continuous_data.csv" #I TAKE THE WORDS FROM THIS CSV, YOU WILL NEED YOUR OWN OBVIOUSLY - MB REFRACTOR TO TAKE TXT FILE IF NEEDED
FALLBACK_C4_JSON = "C4_sentences.json"
WORDS_LIMIT = None

OUT_DIR = Path("./per_layer_word_embeddings")

MODELS = [
    {"name": "JinaEmb-v3", "id": "jinaai/jina-embeddings-v3", "short": "jina_v3", "trc": True, "type": "encoder"},
    # add more if needed
]

# --------------- UTILS ------------------
def clean_mem() -> None:
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats()

def sanitize(text: str) -> str:
    return "".join(ch if ch.isalnum() or ch in "._-" else "_" for ch in text)

def find_subsequence_tokens(big: List[str], small: List[str]) -> Tuple[int, int]:
    n, m = len(big), len(small)
    if m == 0 or m > n:
        return -1, -1
    for i in range(n - m + 1):
        if big[i:i + m] == small:
            return i, i + m
    return -1, -1

# ------------- LOAD MODELS --------------
def load_encoder(model_id: str, trust_remote_code: bool):
    cfg = AutoConfig.from_pretrained(model_id, trust_remote_code=trust_remote_code)
    cfg.output_hidden_states = True
    tok = AutoTokenizer.from_pretrained(model_id, use_fast=True, trust_remote_code=trust_remote_code)
    model = AutoModel.from_pretrained(
        model_id,
        config=cfg,
        trust_remote_code=trust_remote_code,
        torch_dtype=(torch.bfloat16 if (USE_BF16 and torch.cuda.is_available()) else None),
        low_cpu_mem_usage=True,
    ).to(DEVICE).eval()

    if tok.pad_token is None:
        if tok.eos_token is not None:
            tok.pad_token = tok.eos_token
        else:
            tok.add_special_tokens({"pad_token": "[PAD]"})
        try:
            model.resize_token_embeddings(len(tok))
        except Exception:
            pass
    tok.padding_side = "right"
    return tok, model

def load_decoder(model_id: str, trust_remote_code: bool):
    cfg = AutoConfig.from_pretrained(model_id, trust_remote_code=trust_remote_code)
    cfg.output_hidden_states = True
    cfg.use_cache = False

    tok = AutoTokenizer.from_pretrained(model_id, use_fast=True, trust_remote_code=trust_remote_code)
    if tok.pad_token is None:
        if tok.eos_token is not None:
            tok.pad_token = tok.eos_token
        else:
            tok.add_special_tokens({"pad_token": "[PAD]"})
    tok.padding_side = "right"

    common = dict(trust_remote_code=trust_remote_code, device_map="auto", low_cpu_mem_usage=True)
    dtype = torch.bfloat16 if torch.cuda.is_available() else torch.float32
    try:
        model = AutoModelForCausalLM.from_pretrained(model_id, dtype=dtype, **common)
    except TypeError:
        model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=dtype, **common)

    if getattr(model.config, "use_cache", True):
        model.config.use_cache = False

    model.eval()
    return tok, model

# --------- SENTENCE EMBEDDINGS ----------
@torch.inference_mode()
def embed_last_layer_in_sentence(
    word: str,
    tok,
    model,
    pool: str = POOL,
    template: str = SENTENCE_TEMPLATE,
) -> np.ndarray:
    """
    Build a sentence with 'word', run the model, and return a single vector
    pooled over the word's token span **from the last hidden layer only**.
    Returns: np.ndarray of shape (hidden_size,)
    """
    sentence = template.format(word=word)

    enc = tok(
        sentence,
        return_tensors="pt",
        return_offsets_mapping=True,
        return_special_tokens_mask=True,
        add_special_tokens=True,
    )

    token_indices: List[int] = []
    offsets = enc.get("offset_mapping")
    specials = enc.get("special_tokens_mask")

    # try robust char-span mapping
    if offsets is not None:
        offs = offsets[0].tolist()
        spmsk = (specials[0].tolist() if specials is not None else [0] * len(offs))
        a = sentence.lower().find(word.lower())
        if a >= 0:
            b = a + len(word)
            for i, ((x, y), sp) in enumerate(zip(offs, spmsk)):
                if sp == 1 or y <= x:
                    continue
                if not (y <= a or x >= b):  # overlap
                    token_indices.append(i)

    # fallback: token subsequence
    if not token_indices:
        ids_full = tok(sentence, add_special_tokens=True)["input_ids"]
        ids_word = tok(word, add_special_tokens=False)["input_ids"]
        toks_full = tok.convert_ids_to_tokens(ids_full)
        toks_word = tok.convert_ids_to_tokens(ids_word)
        start, end = find_subsequence_tokens(toks_full, toks_word)
        if start >= 0:
            token_indices = list(range(start, end))

    # fallback: interior non-special tokens
    if not token_indices:
        attn = enc["attention_mask"][0].tolist()
        nz = [i for i, m in enumerate(attn) if m == 1]
        token_indices = nz[1:-1] or nz

    # to device
    tens = {k: v.to(DEVICE) for k, v in enc.items() if torch.is_tensor(v)}
    tens.pop("offset_mapping", None)
    tens.pop("special_tokens_mask", None)

    # forward
    if torch.cuda.is_available():
        with torch.autocast("cuda", dtype=torch.bfloat16):
            out = model(**tens, output_hidden_states=True)
    else:
        out = model(**tens, output_hidden_states=True)

    # --- ONLY LAST LAYER ---
    if getattr(out, "hidden_states", None) is not None:
        last = out.hidden_states[-1]         # [1, seq_len, hidden]
    else:
        last = out.last_hidden_state         # [1, seq_len, hidden]

    # pool subwords in the span
    token_vecs = last[0, token_indices, :]    # [k, hidden]
    if pool == "mean":
        v = token_vecs.mean(dim=0)
    elif pool == "first":
        v = token_vecs[0]
    elif pool == "last":
        v = token_vecs[-1]
    else:
        raise ValueError("POOL must be 'mean' | 'first' | 'last'")

    return v.detach().cpu().to(torch.float32).numpy()  # (hidden,)

# --------------- MAIN -------------------
def load_words() -> List[str]:
    if WORDS_CSV and Path(WORDS_CSV).exists():
        df = pd.read_csv(WORDS_CSV)
        words = df.iloc[:, 0].astype(str).tolist()
        return words[:WORDS_LIMIT] if WORDS_LIMIT is not None else words

    if Path(FALLBACK_C4_JSON).exists():
        with open(FALLBACK_C4_JSON, "r", encoding="utf-8") as f:
            data = json.load(f)
        words = list(data.keys())
        return words[:WORDS_LIMIT] if WORDS_LIMIT is not None else words

    raise FileNotFoundError("No WORDS_CSV or FALLBACK_C4_JSON found")

def process_model(model_spec: dict, words: List[str]) -> None:
    name = model_spec["name"]
    model_id = model_spec["id"]
    short = model_spec["short"]
    trust_remote_code = model_spec["trc"]
    model_type = model_spec["type"]

    out_dir = OUT_DIR / short / "sentence_last"  # <-- ONLY LAST LAYER
    out_dir.mkdir(parents=True, exist_ok=True)

    print(f"\n🚀 Loading {name} ({model_id}) → {out_dir}")
    try:
        if model_type == "encoder":
            tok, mdl = load_encoder(model_id, trust_remote_code)
        else:
            tok, mdl = load_decoder(model_id, trust_remote_code)
    except Exception as e:
        print(f"❌ Skipping {name} — load failed: {e}")
        clean_mem()
        return

    total = len(words)
    for i, w in enumerate(words, 1):
        fname = sanitize(w) or f"w_{i}"
        out_path = out_dir / f"{fname}.npy"

        if out_path.exists():
            if i % 200 == 0:
                print(f"  [{i}/{total}] {w} (cached)")
            continue

        try:
            vec = embed_last_layer_in_sentence(w, tok, mdl).astype(np.float16, copy=False)  # (hidden,)
            np.save(out_path, vec)
        except Exception as e:
            print(f"  [{i}/{total}] {w} — sentence (last layer) FAILED: {e}")
            continue

        if i % 50 == 0:
            print(f"  [{i}/{total}] {w} ✓ (hidden={vec.shape[0]})")

    del mdl, tok
    clean_mem()
    print(f"✅ Finished {name} — unloaded.")

def main() -> None:
    words = load_words()
    print(f"📚 Words in this run: {len(words)}")
    OUT_DIR.mkdir(parents=True, exist_ok=True)
    for spec in MODELS:
        process_model(spec, words)
    print("\n✅ All models finished (sentence, last layer only). Output:", OUT_DIR.resolve())

if __name__ == "__main__":
    try:
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.set_float32_matmul_precision("high")
    except Exception:
        pass
    main()
